<a href="https://colab.research.google.com/github/danielicapui/machine_learning/blob/main/trabalho_final_sistema_de_recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import sklearn.metrics.pairwise as pw
from sklearn import preprocessing

In [ ]:
 
print("grupo:Daniel Lucas,Jeferson Ximenes,Marcos Fonseca,")
#df = pd.read_csv('winemag-data-130k-v2.csv')

In [ ]:
 #Eu fiz upload no git para não precisar de autenticação
arquivo='https://github.com/danielicapui/machine_learning/blob/main/winemag-data-130k-v2.csv.zip?raw=true'
df=pd.read_csv(arquivo,compression='zip',header=0,sep=',',quotechar='"')
#df=pd.read_csv('/content/drive/MyDrive/winemag-data-130k-v2.csv')
df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [ ]:
df.shape

(129971, 14)

In [ ]:
df = df.drop(['Unnamed: 0', 'taster_twitter_handle', 'region_2', 'designation'], axis = 1)

In [ ]:
df = df.dropna()

In [ ]:
duplicated_rows = df[df.duplicated()]
 
index_duplicated = duplicated_rows.index
 
df = df.drop(index_duplicated)
df = df.reset_index()

In [ ]:
df.head()

,index,country,description,points,price,province,region_1,taster_name,title,variety,winery
0,2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
4,6,Italy,"Here's a bright, informal red that opens with ...",87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo


In [ ]:
df.shape

(70657, 11)

## Content base recommendation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df['len'] = df['description'].apply(lambda x: len(x.split(' ')))
wine_df = df[df['len'] >= 50]

In [ ]:
wine_df

,index,country,description,points,price,province,region_1,taster_name,title,variety,winery,len
14,19,US,"Red fruit aromas pervade on the nose, with cig...",87,32.0,Virginia,Virginia,Alexander Peartree,Quiévremont 2012 Meritage (Virginia),Meritage,Quiévremont,55
24,29,US,Clarksburg is becoming a haven for Chenin Blan...,86,16.0,California,Clarksburg,Virginie Boone,Clarksburg Wine Company 2010 Chenin Blanc (Cla...,Chenin Blanc,Clarksburg Wine Company,50
50,81,Spain,"Bland, mature aromas of white fruits are on th...",86,16.0,Galicia,Rías Baixas,Michael Schachner,Spyro 2014 Albariño (Rías Baixas),Albariño,Spyro,52
64,97,US,A wisp of bramble extends a savory tone from n...,88,20.0,New York,Finger Lakes,Anna Lee C. Iijima,Heron Hill 2015 Ingle Vineyard Riesling (Finge...,Riesling,Heron Hill,53
74,108,US,"Lots of spearmint, coyote mint, hot licorice, ...",87,26.0,California,Santa Clara Valley,Matt Kettmann,Martin Ranch 2014 J.D. Hurley Zinfandel (Santa...,Zinfandel,Martin Ranch,53
...,...,...,...,...,...,...,...,...,...,...,...,...
70637,129938,Argentina,Compared to the regular 2006 Malbec from Chaka...,91,25.0,Mendoza Province,Luján de Cuyo,Michael Schachner,Chakana 2006 Reserve Malbec (Luján de Cuyo),Malbec,Chakana,58
70641,129945,US,Hailing from one of the more popular vineyards...,90,20.0,California,Santa Ynez Valley,Matt Kettmann,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,Birichino,66
70644,129949,US,There's no bones about the use of oak in this ...,90,35.0,California,Napa Valley,Virginie Boone,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs,50
70648,129957,Spain,Lightly baked berry aromas vie for attention w...,90,17.0,Northern Spain,Rioja,Michael Schachner,Viñedos Real Rubio 2010 Crianza (Rioja),Tempranillo Blend,Viñedos Real Rubio,62


In [ ]:
wines_desc = wine_df['description'].fillna('').to_list()
wines_desc

['Red fruit aromas pervade on the nose, with cigar box and menthol notes riding in the back. The palate is slightly restrained on entry, but opens up to riper notes of cherry and plum specked with crushed pepper. This blend of Merlot, Cabernet Sauvignon and Cabernet Franc is approachable now and ready to be enjoyed.',
 'Clarksburg is becoming a haven for Chenin Blanc in California. This bottling, using fruit sourced from several vineyards in the area, is balanced with just a trace of sweetness in the background (1% residual sugar). This is crisp, straightforward and blessed with notes of pear and lime. Drink it cold.',
 'Bland, mature aromas of white fruits are on the weak side for Albariño. This feels round and oily, while briny leesy flavors lack pop and precision. A grabby aged finish is short on the free flow that this grape and region are known for. Drink immediately or wait for a newer vintage.',
 'A wisp of bramble extends a savory tone from nose to finish on this otherwise brig

In [ ]:
vectorizer = CountVectorizer(stop_words = 'english')

In [ ]:
bow = vectorizer.fit_transform(wines_desc)
bow_vector = bow.sum(axis = 0).tolist()[0]

In [ ]:
top_words = pd.DataFrame(data = bow_vector, index = vectorizer.get_feature_names(), columns = ['counter'])
top_words = top_words.nlargest(100, 'counter')
top_words

,counter
wine,9757
flavors,7010
fruit,6756
palate,6035
black,5404
...,...
pear,781
wines,773
hints,769
wood,768


### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()

wine_df['title_id'] = le.fit_transform(wine_df['title'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
content_df = wine_df[['title_id', 'title', 'description']]
content_df = content_df.set_index('title_id')
content_df.reset_index(inplace = True)

In [ ]:
content_df

,title_id,title,description
0,10774,Quiévremont 2012 Meritage (Virginia),"Red fruit aromas pervade on the nose, with cig..."
1,3531,Clarksburg Wine Company 2010 Chenin Blanc (Cla...,Clarksburg is becoming a haven for Chenin Blan...
2,12154,Spyro 2014 Albariño (Rías Baixas),"Bland, mature aromas of white fruits are on th..."
3,6768,Heron Hill 2015 Ingle Vineyard Riesling (Finge...,A wisp of bramble extends a savory tone from n...
4,8973,Martin Ranch 2014 J.D. Hurley Zinfandel (Santa...,"Lots of spearmint, coyote mint, hot licorice, ..."
...,...,...,...
14307,2610,Chakana 2006 Reserve Malbec (Luján de Cuyo),Compared to the regular 2006 Malbec from Chaka...
14308,1337,Birichino 2013 Jurassic Park Vineyard Old Vine...,Hailing from one of the more popular vineyards...
14309,5854,Flora Springs 2013 Barrel Fermented Chardonnay...,There's no bones about the use of oak in this ...
14310,13650,Viñedos Real Rubio 2010 Crianza (Rioja),Lightly baked berry aromas vie for attention w...


In [ ]:
vectorizer = TfidfVectorizer(vocabulary = top_words.index)
bow = vectorizer.fit_transform(wines_desc)

In [ ]:
bow_array = bow.toarray()
bow.shape

(14312, 100)

In [ ]:
text_features = pd.DataFrame(data = bow_array, columns = vectorizer.get_feature_names(), index = content_df.title_id)
text_features

,wine,flavors,fruit,palate,black,aromas,finish,drink,cherry,tannins,nose,acidity,ripe,red,blend,notes,oak,spice,cabernet,dark,plum,berry,rich,blackberry,vineyard,fresh,pepper,fruits,shows,chocolate,white,sauvignon,vanilla,dried,raspberry,texture,dry,apple,light,years,...,herb,soft,structure,bright,citrus,concentrated,fine,bit,coffee,tart,balance,cassis,tobacco,leather,spicy,best,crisp,dense,style,savory,tannic,green,franc,earth,pinot,flavor,character,complex,tight,hint,herbs,smooth,structured,earthy,malbec,pear,wines,hints,wood,elegant
title_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10774,0.000000,0.000000,0.149240,0.147811,0.0,0.162917,0.000000,0.000000,0.185157,0.0,0.191516,0.000000,0.000000,0.215862,0.211293,0.438260,0.000000,0.000000,0.478568,0.0,0.230155,0.000000,0.0,0.000000,0.000000,0.0,0.246292,0.000000,0.0,0.000000,0.00000,0.257847,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.3017,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3531,0.000000,0.000000,0.207801,0.000000,0.0,0.000000,0.000000,0.243823,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.305116,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.414211,0.0,0.0,0.00000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.435490,0.0,0.0,0.0,0.0
12154,0.000000,0.247481,0.000000,0.000000,0.0,0.288162,0.295920,0.309729,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.443645,0.0,0.000000,0.46782,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6768,0.160774,0.174721,0.000000,0.184578,0.0,0.000000,0.417838,0.218668,0.000000,0.0,0.239155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.329907,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.357882,0.0,0.0,0.000000,0.0,0.0,0.0,0.366478,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.37567,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
8973,0.188355,0.000000,0.436666,0.000000,0.0,0.000000,0.000000,0.000000,0.270879,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.321906,0.000000,0.0,0.000000,0.000000,0.0,0.350699,0.000000,0.0,0.360317,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610,0.159368,0.173193,0.000000,0.000000,0.0,0.201663,0.207092,0.000000,0.229192,0.0,0.000000,0.000000,0.250474,0.267200,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.289975,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.313884,0.00000,0.000000,0.000000,0.0,0.322277,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000

In [ ]:
wine_idx = 0
n_wines = 10

In [ ]:
text_similarity = pd.DataFrame(pw.cosine_similarity(text_features.iloc[wine_idx:wine_idx+1], 
                                                  text_features[wine_idx+1:]).T,
                             columns = ['similarity'], index = text_features.index[wine_idx+1:])
text_similarity

,similarity
title_id,
3531,0.164732
12154,0.046946
6768,0.073085
8973,0.204066
11078,0.190299
...,...
2610,0.132969
1337,0.125104
5854,0.000000


In [ ]:
recommendation_idx_text = text_similarity.nlargest(n_wines, 'similarity')
(content_df[content_df['title_id'].isin(recommendation_idx_text.index)])['title']

795       Toccata 2013 Classico Red (Santa Barbara County)
4780           Joullian NV Retro Rouge Red (Carmel Valley)
6428                        Hedges 2013 Red (Red Mountain)
7580     Carhartt Vineyard 2012 Fourplay Red (Santa Yne...
8481     Fattoria Montellori 2010 Salamartano Red (Tosc...
9299          Northstar 2014 Red (Walla Walla Valley (WA))
11441    Fielding Hills 2009 RiverBend Vineyard Tribute...
11669    Buttonwood 2013 Classic Cuvée Red (Santa Ynez ...
12555    Double Eagle 2012 Grieve Family Vineyard Red (...
14055    Charles & Charles 2015 Merlot Blend Red (Colum...
Name: title, dtype: object

In [ ]:
def wine_recommender(wine_idx):
  display(f'Get wine recommendation for {content_df.iloc[wine_idx, 1]}')
  wine_similarity = pd.DataFrame(pw.cosine_similarity(text_features.iloc[wine_idx: wine_idx + 1], text_features).T,
                             columns = ['similarity'], index = text_features.index)
  
  recommendation_idx = wine_similarity.nlargest(n_wines + 1, 'similarity')
  return (content_df [ content_df['title_id'].isin(recommendation_idx.index)])['title'].tolist()

In [ ]:
wine_recommender(10)

'Get wine recommendation for Alta Colina 2012 Old 900 Syrah (Paso Robles)'

['Alta Colina 2012 Old 900 Syrah (Paso Robles)',
 'Beauregard 2013 Beaureagard Ranch Chardonnay (Ben Lomond Mountain)',
 'Domaine Vacheron 2015  Sancerre',
 'Château Meyney 2010 La Chapelle de Meyney  (Saint-Estèphe)',
 'Kathryn Kennedy 2013 Merlot (Santa Cruz Mountains)',
 'Château Ponzac 2015 Patiemment Malbec (Cahors)',
 'Loring Wine Company 2015 Huber Vineyard Dornfelder (Sta. Rita Hills)',
 'Golden Triangle 2013 Cabernet Sauvignon (Paso Robles Willow Creek District)',
 'Patrimony 2013 Cabernet Sauvignon (Adelaida District)',
 'Joseph Drouhin 2012 Petit Monts Premier Cru  (Vosne-Romanée)',
 'Château Latour 2011  Pauillac']

## Collaborative Filtering

In [ ]:
le = preprocessing.LabelEncoder()

df['taster_id'] = le.fit_transform(df['taster_name'])
df['title_id'] = le.fit_transform(df['title'])

colab_df = df[['taster_id','title_id', 'points']]

In [ ]:
colab_df = colab_df.reset_index()
colab_df = colab_df.drop(['index'], axis = 1)

In [ ]:
colab_df

,taster_id,title_id,points
0,12,53652,87
1,0,59864,87
2,12,60794,87
3,11,61268,87
4,8,62369,87
...,...,...,...
70652,13,23531,90
70653,12,18279,90
70654,13,22770,90
70655,13,23217,90


In [ ]:
matriz_utilidade_provador_vinho = pd.pivot_table(colab_df, index = 'taster_id', columns = 'title_id', values = 'points')
matriz_utilidade_provador_vinho

title_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,69910,69911,69912,69913,69914,69915,69916,69917,69918,69919,69920,69921,69922,69923,69924,69925,69926,69927,69928,69929,69930,69931,69932,69933,69934,69935,69936,69937,69938,69939,69940,69941,69942,69943,69944,69945,69946,69947,69948,69949
taster_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,89.0,84.0,86.0,91.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
matriz_utilidade_provador_vinho_com_zeros = matriz_utilidade_provador_vinho.fillna(0)

matriz_similaridade_coseno = pd.DataFrame(pw.cosine_similarity(matriz_utilidade_provador_vinho_com_zeros),
                                          index = matriz_utilidade_provador_vinho_com_zeros.index,
                                          columns = matriz_utilidade_provador_vinho_com_zeros.index)

matriz_similaridade_coseno

taster_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
taster_id,,,,,,,,,,,,,,,,,
0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,0.000000,0.001759,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001331,0.000000
2,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000481,0.0,0.0,0.0,0.0,0.000000,0.003248,0.000000,0.000000,0.000000
3,0.0,0.001759,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.008155,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.004404
5,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.002083
7,0.0,0.000000,0.000481,0.000000,0.008155,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,0.000313,0.000000,0.000000,0.000000
8,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
provadores_similares_12 = matriz_similaridade_coseno[12].sort_values(ascending = False)[1:11]
provadores_similares_12

taster_id
14    0.001418
13    0.000378
16    0.000000
6     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
8     0.000000
Name: 12, dtype: float64

In [ ]:
pontos_provadores_similares = matriz_utilidade_provador_vinho_com_zeros.loc[provadores_similares_12.index]
pontos_provadores_similares

title_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,69910,69911,69912,69913,69914,69915,69916,69917,69918,69919,69920,69921,69922,69923,69924,69925,69926,69927,69928,69929,69930,69931,69932,69933,69934,69935,69936,69937,69938,69939,69940,69941,69942,69943,69944,69945,69946,69947,69948,69949
taster_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,0.0,0.0,90.0,90.0,0.0,0.0,0.0,90.0,0.0,0.0,90.0,88.0,87.0,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.0,0.0,0.0,0.0,92.0,89.0,0.0,88.0,92.0,89.0,93.0,89.0,91.0,91.0,91.0,91.0,91.0,89.0,90.0,90.0,91.0,89.0,89.0,88.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,93.0,93.0,93.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,89.0,84.0,86.0,91.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rank_vinhos = np.average(pontos_provadores_similares, axis = 0, weights = provadores_similares_12)
rank_vinhos

array([ 0.        ,  0.        ,  0.        , ..., 70.24854378,
       69.4592343 ,  0.        ])

In [ ]:
vinhos_provador_12 = colab_df[colab_df['taster_id'] == 12]
vinhos_provador_12

,taster_id,title_id,points
0,12,53652,87
2,12,60794,87
16,12,603,87
25,12,27433,86
26,12,33272,86
...,...,...,...
70626,12,48224,91
70629,12,54296,91
70630,12,61176,91
70635,12,65029,91


In [ ]:
rank_vinhos_df = pd.DataFrame({'rank': rank_vinhos}, index = pontos_provadores_similares.columns)

usuario12_provou = rank_vinhos_df.index.isin(vinhos_provador_12.title_id)
rank_vinhos_df = rank_vinhos_df.loc[~usuario12_provou]

top10_provador12 = rank_vinhos_df['rank'].sort_values(ascending = False).iloc[0:10].index.to_list()
top10_provador12

[34365, 34370, 12008, 34367, 12796, 34350, 12790, 12000, 34366, 37248]

In [ ]:
id_vinhos_preferidos_provador12 = vinhos_provador_12.sort_values(ascending = False, by = 'points').iloc[:10, 1]

vinhos_preferidos_provador12 = df[df['title_id'].isin(id_vinhos_preferidos_provador12)]
vinhos_preferidos_provador12['title']

946      Quilceda Creek 2008 Cabernet Sauvignon (Columb...
949      Pirouette 2008 Red Wine Red (Columbia Valley (...
27249    Cayuse 2009 En Chamberlin Vineyard Syrah (Wall...
27548    Cayuse 2011 En Chamberlin Vineyard Syrah (Wall...
27549    Cayuse 2011 En Cerise Vineyard Syrah (Walla Wa...
46539    Cayuse 2008 En Cerise Vineyard Syrah (Walla Wa...
62435    Charles Smith 2006 Royal City Syrah (Columbia ...
67469    Cayuse 2008 Bionic Frog Syrah (Walla Walla Val...
68653    Figgins 2009 Estate Red (Walla Walla Valley (WA))
68654    Quilceda Creek 2009 Cabernet Sauvignon (Columb...
Name: title, dtype: object

In [ ]:
id_vinhos_odiados_provador12 = vinhos_provador_12.sort_values(by = 'points').iloc[:10, 1]

vinhos_odiados_provador12 = df[df['title_id'].isin(id_vinhos_odiados_provador12)]
vinhos_odiados_provador12['title']

7609     Naches Heights 2006 Pinot Gris (Columbia Valle...
8350                Del Rio 2007 Pinot Noir (Rogue Valley)
8351     Namasté Vineyards 2006 Serenity Vineyard Chard...
15514    Cathedral Ridge 2013 Cabernet Sauvignon (Colum...
28460    Mount Baker Vineyards 2005 Barrel Select Merlo...
31679    Cathedral Ridge 2008 McDuffee Vineyard Reserve...
31681    Château Bianca 2008 Pinot Noir (Willamette Val...
31682             Agate Ridge 2008 Grenache (Rogue Valley)
31683    Cathedral Ridge 2007 Cabernet Sauvignon (Colum...
31684    Agate Ridge 2007 Cascade Terrace Red Red (Rogu...
Name: title, dtype: object

In [ ]:
vinhos_sugeridos_provador12 = df[df['title_id'].isin(top10_provador12)]
vinhos_sugeridos_provador12['title']

9736     Hors Categorie 2014 Syrah (Walla Walla Valley ...
17337    Horsepower 2013 The Tribe Vineyard Syrah (Wall...
26329    Charles Smith 2012 Royal City Syrah (Columbia ...
60458    K Vintners 2012 The Hustler Syrah (Walla Walla...
60460    Charles Smith 2013 Royal City Northridge Viney...
65004    Horsepower 2012 Sur Echalas Vineyard Grenache ...
65005    Cayuse 2012 God Only Knows Grenache (Walla Wal...
65006    Cayuse 2012 Bionic Frog Syrah (Walla Walla Val...
65007    Horsepower 2012 Sur Echalas Vineyard Syrah (Wa...
65008    Horsepower 2012 The Tribe Vineyard Syrah (Wall...
Name: title, dtype: object

#Recomendation for popularity

In [ ]:
arquivo='https://github.com/danielicapui/machine_learning/blob/main/winemag-data-130k-v2.csv.zip?raw=true'
df=pd.read_csv(arquivo,compression='zip',header=0,sep=',',quotechar='"')
#df=pd.read_csv('/content/drive/MyDrive/winemag-data-130k-v2.csv')
df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [ ]:
wine_df=df.groupby("title"). filter(lambda x: x['points'].count()>= 4)
wine_df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
406,406,France,"A creamy, caramel- and pear-flavored wine, kee...",Brut Mosaïque,85,30.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Jacquart NV Brut Mosaïque (Champagne),Champagne Blend,Jacquart
959,959,France,Partially from Grand Cru vineyards in the Mont...,Brut Mosaïque,87,40.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Jacquart NV Brut Mosaïque (Champagne),Champagne Blend,Jacquart
2179,2179,Spain,"This aerated, yellow-tinted brut nature smells...",Glaç Brut Nature,90,31.0,Catalonia,Cava,NaN,Michael Schachner,@wineschach,Maria Casanovas NV Glaç Brut Nature Sparkling ...,Sparkling Blend,Maria Casanovas
3209,3209,US,"Creamy, lush and somewhat robust, this dry spa...",Sonoma Brut,90,22.0,California,Sonoma County,Sonoma,Virginie Boone,@vboone,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Sparkling Blend,Gloria Ferrer
3500,3500,US,This frankly sweet sparkler smells like toaste...,Sweet Cuvée,88,14.0,California,California,California Other,Jim Gordon,@gordone_cellars,Korbel NV Sweet Cuvée Sparkling (California),Sparkling Blend,Korbel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129320,129320,France,"Named after Léonie Duchêne, one of the founde...",Cuvée Léonie Brut,90,35.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Canard-Duchêne NV Cuvée Léonie Brut (Champagne),Champagne Blend,Canard-Duchêne
129469,129469,France,"Alain Thiénot's non vintage is ripe, rich and ...",Brut,88,NaN,Champagne,Champagne,NaN,Roger Voss,@vossroger,Thiénot NV Brut (Champagne),Champagne Blend,Thiénot
129502,129502,France,This wine is ripe and immediately drinkable. I...,Cuvée Royale Brut,88,45.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Joseph Perrier NV Cuvée Royale Brut (Champagne),Champagne Blend,Joseph Perrier
129694,129694,US,This is one of Chandon's sweeter bubblies. It ...,Brut Classic,87,22.0,California,California,California Other,NaN,NaN,Chandon NV Brut Classic Sparkling (California),Sparkling Blend,Chandon


In [ ]:
points_df = pd.DataFrame(wine_df.groupby('title').points.mean())
points_df

,points
title,
A.R. Lenoble NV Intense Brut (Champagne),89.500000
Bailly-Lapierre NV Brut (Crémant de Bourgogne),90.333333
Besserat de Bellefon NV Cuvée des Moines Brut Rosé (Champagne),87.000000
Billecart-Salmon NV Brut Rosé (Champagne),91.500000
Bodegas Dios Baco S.L. NV Amontillado Sherry (Jerez),90.750000
...,...
Thiénot NV Brut Rosé (Champagne),87.800000
Vignerons des Pierres Dorées 2015 Salamandre d'Or (Coteaux Bourguignons),86.000000
Villa Sandi NV Extra Dry (Conegliano Valdobbiadene Prosecco Superiore),87.750000


In [ ]:
points_df['points_counts'] =wine_df.groupby('title').points.count()
points_df.sort_values(by = 'points_counts', ascending = False)

,points,points_counts
title,,
Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma County),89.000000,11
Korbel NV Brut Sparkling (California),86.888889,9
Segura Viudas NV Extra Dry Sparkling (Cava),85.625000,8
Gloria Ferrer NV Blanc de Noirs Sparkling (Carneros),88.142857,7
Ruinart NV Brut Rosé (Champagne),89.714286,7
...,...,...
Korbel NV Extra Dry Sparkling (California),86.000000,4
Korbel NV Sweet Cuvée Sparkling (California),86.500000,4
Lanson NV Black Label Brut (Champagne),89.750000,4


In [ ]:
#10 produtos mais avaliados
points_df.head(10)

,points,points_counts
title,,
A.R. Lenoble NV Intense Brut (Champagne),89.500000,4
Bailly-Lapierre NV Brut (Crémant de Bourgogne),90.333333,6
Besserat de Bellefon NV Cuvée des Moines Brut Rosé (Champagne),87.000000,4
Billecart-Salmon NV Brut Rosé (Champagne),91.500000,4
Bodegas Dios Baco S.L. NV Amontillado Sherry (Jerez),90.750000,4
Boizel NV Brut Réserve (Champagne),88.666667,6
Breathless NV Brut Sparkling (North Coast),88.600000,5
Canard-Duchêne NV Authentic Brut (Champagne),87.500000,4
Canard-Duchêne NV Cuvée Léonie Brut (Champagne),89.750000,4


In [ ]:
#Contagem 4,mg=valor da media global, mi=valor minjmo
mg=points_df['points'].mean()
mi=points_df.points_counts.min()

In [ ]:
 def rank (df, mi=mi, mg=mg):
    v = df['points_counts']
    R = df['points']
    return(v/(v+mi) * R) +(mi/(mi+v) *mg)

In [ ]:
#usando apply para aplicar a função
points_df['ranking']=points_df.apply(rank,axis=1)
points_df.sort_values(by='ranking',ascending=False)

,points,points_counts,ranking
title,,,
Roederer Estate NV Brut Rosé Sparkling (Anderson Valley),91.800000,5,90.199496
Domaine Vacheron 2015 Sancerre,92.000000,4,90.099433
Ruinart NV Blanc de Blancs Brut Chardonnay (Champagne),91.750000,4,89.974433
Charles Heidsieck NV Brut Réserve (Champagne),91.750000,4,89.974433
Louis Roederer NV Brut Premier (Champagne),91.750000,4,89.974433
...,...,...,...
Korbel NV Sweet Rosé Sparkling (California),86.000000,6,86.879547
Freixenet NV Carta Nevada Brut Sparkling (Cava),85.500000,4,86.849433
Freixenet NV Cordon Negro Extra Dry Sparkling (Cava),85.600000,5,86.755052


In [ ]:
points_df.ranking.head(20)

title
A.R. Lenoble  NV Intense Brut  (Champagne)                              88.849433
Bailly-Lapierre NV Brut  (Crémant de Bourgogne)                         89.479547
Besserat de Bellefon NV Cuvée des Moines Brut Rosé  (Champagne)         87.599433
Billecart-Salmon NV Brut Rosé  (Champagne)                              89.849433
Bodegas Dios Baco S.L. NV Amontillado Sherry (Jerez)                    89.474433
Boizel NV Brut Réserve  (Champagne)                                     88.479547
Breathless NV Brut Sparkling (North Coast)                              88.421719
Canard-Duchêne NV Authentic Brut  (Champagne)                           87.849433
Canard-Duchêne NV Cuvée Léonie Brut  (Champagne)                        88.974433
Cave de Ribeauvillé NV Giersberger Brut Sparkling (Crémant d'Alsace)    87.224433
Chandon NV Blanc de Noirs Sparkling (California)                        88.099433
Chandon NV Brut Classic Sparkling (California)                          87.224433
Chandon NV